<a href="https://colab.research.google.com/github/talhaanwarch/ODIR2019/blob/master/attention_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODO
*cross check the implementation by some one expert

#heling material
https://github.com/ivashnyov/ODIR/issues/1  
https://github.com/nnnnew/ODIR  
https://github.com/DecentMakeover/odir_winningsubmission

# read data from directory

In [0]:
#connect drive with colab notebook
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# #save variables in drive, so next we can load them directly
import numpy as np
#for mam
# XL=np.load('/content/drive/My Drive/thesis/XL.npy')
# XR=np.load('/content/drive/My Drive/thesis/XR.npy')
# y=np.load('/content/drive/My Drive/thesis/y.npy')

XL=np.load('/content/drive/My Drive/ghina/thesis/XL.npy')
XR=np.load('/content/drive/My Drive/ghina/thesis/XR.npy')
y=np.load('/content/drive/My Drive/ghina/thesis/y.npy')

In [0]:
#split data
from sklearn.model_selection import train_test_split
XL_train, XL_val, yL_train, yL_val = train_test_split(XL, y, test_size=0.2, random_state=42)
XR_train, XR_val, yR_train, yR_val = train_test_split(XR, y, test_size=0.2, random_state=42)

In [0]:
len(yL_val),len(yR_val)

(700, 700)

In [0]:
sum(XL_val==XR_val).any()

True

In [0]:
sum(yL_val==yR_val)

array([700, 700, 700, 700, 700, 700, 700, 700])

#Date generator

In [0]:
BATCH_SIZE=32

In [0]:
#data generator
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               vertical_flip=False,
                               fill_mode='nearest'# Fill in missing pixels with the nearest filled value
                                )
val_datagen=ImageDataGenerator(rescale=1/255)

In [0]:
def multi_train_gen(gen,XR_train,XL_train,yR_train,yL_train):
  XR_generator = train_datagen.flow(XR_train, yR_train, batch_size=BATCH_SIZE)
  XL_generator = train_datagen.flow(XL_train, yL_train, batch_size=BATCH_SIZE)
  while True:
    X1i = XR_generator.next()
    X2i = XL_generator.next()
    yield [X1i[0], X2i[0]], X2i[1] 

In [0]:
input_gen=multi_train_gen(train_datagen,XR_train,XL_train,yR_train,yL_train)    

In [0]:
def multi_val_gen(gen,XR_val,XL_val,yR_val,yL_val):
  vR_generator = val_datagen.flow(XR_val, yR_val, batch_size=BATCH_SIZE)
  vL_generator = val_datagen.flow(XL_val, yL_val, batch_size=BATCH_SIZE)

  while True:
    X1i = vR_generator.next()
    X2i = vL_generator.next()
    yield [X1i[0], X2i[0]], X2i[1] 

In [0]:
val_gen=multi_val_gen(val_datagen,XR_val,XL_val,yR_val,yL_val)    

In [0]:
#steps_train = XR_generator.n//XR_generator.batch_size
#steps_valid = vR_generator.n//vR_generator.batch_size


In [0]:
steps_train=(len(XL_train)/BATCH_SIZE)
steps_train

87.5

In [0]:
steps_valid=(len(XL_val)/BATCH_SIZE)
steps_valid

21.875

#RESNET 50

following idea is borrowed from following link. 
https://github.com/nnnnew/ODIR/blob/master/train/2_pretrained_model_and_1_classifier.ipynb  
Data Generator is combined with multiinput model .
Thanks to https://github.com/keras-team/keras/issues/8130#issuecomment-336855177  
default input size for resent is `224x224`  
https://keras.io/applications/#resnet

In [0]:
from keras.models import Sequential,Model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

In [0]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [0]:
import keras
left_input=Input(shape=XL.shape[1::])
right_input=Input(shape=XR.shape[1::])

left_model = InceptionV3(include_top=False,input_tensor=left_input)
# for layer in left_model.layers:
#         layer.name = layer.name + '_left'
#         layer.trainable = True


right_model = InceptionV3(include_top=False,input_tensor=right_input)
# for layer in right_model.layers:
#         layer.name = layer.name + '_right'
#         layer.trainable = True


87916544/87910968 [==============================] - 1s 0us/step


In [0]:
pt_depth = right_model.get_output_shape_at(0)[-1]
print(pt_depth)

left_bn = BatchNormalization()(left_model.output)
right_bn = BatchNormalization()(right_model.output)


2048


In [0]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda
def attention(bn):
  attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(bn)
  attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
  attn_layer = LocallyConnected2D(1, kernel_size = (1,1), padding = 'valid', activation = 'sigmoid')(attn_layer)
  up_c2_w = np.ones((1, 1, 1, pt_depth))
  up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', activation = 'linear', use_bias = False, weights = [up_c2_w])
  up_c2.trainable = False
  attn_layer = up_c2(attn_layer)
  mask_features = multiply([attn_layer, bn])
  gap_features = GlobalAveragePooling2D()(mask_features)
  gap_mask = GlobalAveragePooling2D()(attn_layer)
  # to account for missing values from the attention model
  gap = Lambda(lambda x: x[0]/x[1])([gap_features, gap_mask])
  gap_dr = Dropout(0.5)(gap)
  dr_steps = Dropout(0.25)(Dense(1024, activation = 'elu')(gap_dr))
  return dr_steps

In [0]:
left_atten=attention(left_bn)
right_atten=attention(right_bn)

In [0]:
for layer in right_model.layers:
        layer.name = layer.name + '_right'
for layer in left_model.layers:
  layer.name = layer.name + '_left'

In [0]:

x = keras.layers.concatenate([left_atten, right_atten])


out = keras.layers.Dense(8, activation='sigmoid')(x)
model = keras.models.Model(inputs=[left_input, right_input], outputs=out)
#https://www.kaggle.com/xhlulu/aptos-2019-densenet-keras-starter

In [0]:
from keras.optimizers import Adam,RMSprop
opt = RMSprop(lr=1e-3,decay=0.2)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[macro_f1])
#model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['macro_f1'])
#https://github.com/ashrefm/multi-label-soft-f1/blob/master/Multi-Label%20Image%20Classification%20in%20TensorFlow%202.0.ipynb

In [0]:
history=model.fit_generator(generator=input_gen,
                        steps_per_epoch=steps_train,
                        validation_data=val_gen,
                        validation_steps=steps_valid,
                        epochs=30,
                        class_weight=[0.5,0.5,1.5,1.5,1.5,1.5,1.5,0.7]
                        )

Epoch 1/30
88/87 [==============================] - 61s 690ms/step - loss: 0.2867 - macro_f1: 0.3022 - val_loss: 0.3106 - val_macro_f1: 0.2943
Epoch 2/30
88/87 [==============================] - 58s 663ms/step - loss: 0.2875 - macro_f1: 0.3199 - val_loss: 0.3086 - val_macro_f1: 0.2705
Epoch 3/30
88/87 [==============================] - 58s 659ms/step - loss: 0.2883 - macro_f1: 0.3099 - val_loss: 0.3091 - val_macro_f1: 0.2763
Epoch 4/30
88/87 [==============================] - 58s 654ms/step - loss: 0.2895 - macro_f1: 0.3249 - val_loss: 0.3065 - val_macro_f1: 0.2824
Epoch 5/30
88/87 [==============================] - 58s 658ms/step - loss: 0.2818 - macro_f1: 0.3263 - val_loss: 0.3083 - val_macro_f1: 0.2822
Epoch 6/30
88/87 [==============================] - 58s 658ms/step - loss: 0.2809 - macro_f1: 0.3322 - val_loss: 0.3049 - val_macro_f1: 0.2780
Epoch 7/30
88/87 [==============================] - 58s 655ms/step - loss: 0.2868 - macro_f1: 0.3115 - val_loss: 0.3058 - val_macro_f1: 0.2804

KeyboardInterrupt: ignored

In [0]:
from sklearn.metrics import classification_report

In [0]:

y_pred = model.predict_generator(val_gen,steps=steps_valid)

In [0]:
y_pred.shape,yL_val.shape

((700, 8), (700, 8))

In [0]:
y_pred[y_pred>=0.5] = 1
y_pred[y_pred<0.5] = 0
y_pred=y_pred.astype(int)

In [0]:
from sklearn.metrics import f1_score
print(classification_report(yL_val, y_pred))

              precision    recall  f1-score   support

           0       0.29      0.42      0.35       209
           1       0.47      0.15      0.23       229
           2       0.00      0.00      0.00        33
           3       0.12      0.12      0.12        43
           4       0.00      0.00      0.00        34
           5       0.00      0.00      0.00        31
           6       0.00      0.00      0.00        33
           7       0.42      0.05      0.08       218

   micro avg       0.29      0.17      0.21       830
   macro avg       0.16      0.09      0.10       830
weighted avg       0.32      0.17      0.18       830
 samples avg       0.18      0.18      0.18       830



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


normal (N), diabetes (D), glaucoma (G), cataract (C), AMD (A), hypertension (H), myopia (M) and other diseases/abnormalities (O) 

In [0]:
lab=['N','D','G','C','A','H','M','O']

In [0]:
from sklearn.metrics import confusion_matrix,f1_score
f=np.zeros(8)
for i,j in zip(range(8),lab):
  f1=f1_score(yL_val[:,i],y_pred[:,i])
  f[i]=f1
  print(j, ":",f1 )

N : 0.33073929961089493
D : 0.19076923076923077
G : 0.0
C : 0.05194805194805195
A : 0.0
H : 0.0
M : 0.10714285714285714
O : 0.08695652173913043


In [0]:
f.sum()/len(f)

0.09594449515127065

In [0]:
keras.backend.clear_session()